# Baseline model on final data (Michael)

## Setup

In [1]:
# import the usual suspects / basics
import time; full_run_time_start = time.time() # start timing exec right away
import pandas as pd
import numpy as np
import pickle
from scipy import sparse
import re
import os

# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, f1_score,\
    accuracy_score, precision_score, recall_score, confusion_matrix

# display all df columns (default is 20)
pd.options.display.max_columns = None

## Utility functions for testing models and tracking results

In [2]:
# empty df for storing results
test_results = pd.DataFrame(columns=['model_name',
                                'model_params',
                                'data_desc',
                                'data_size',
                                'features_no',
                                'f1',
                                'acc',
                                'recall',
                                'prec',
                                'roc_auc',
                                'cf_matrix',
                                'train_time',
                                'notes'])

def test_model(model, model_name, model_params, data_desc, X, y, notes=''):
    '''
    test_model(model, model_params, data_desc, X, y, notes='')
    
    Parameters:
    -----------
    model: instance of model to test
    model_name: name of model
    model_params: dict of (hyper)parameters passed to model
    data_desc: description of dataset (preprocessing steps etc.)
    X: feature array 
    y: target/label array
    notes: additional notes (default: empty string)
    '''

    # Split data using default of 75% for train, 25% for test.
    # Make sure test data has same toxic/nontoxic ratio as train data by
    # using stratify parameter.
    X_train, X_test, y_train, y_test =\
        train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    
    # train model and time execution
    train_time_start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - train_time_start
    train_time_str = f'{int(train_time // 60)}m {round(train_time % 60)}s'

    # Make predictions on test set
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:,1]

    return {'model_name': model_name,
            'model_params': model_params,
            'data_desc': data_desc,
            'data_size': X.shape[0],
            'features_no': X.shape[1],
            'f1': round(f1_score(y_test, y_pred), 5),
            'acc': round(accuracy_score(y_test, y_pred), 5),
            'recall': round(recall_score(y_test, y_pred), 5),
            'prec': round(precision_score(y_test, y_pred), 5),
            'roc_auc': round(roc_auc_score(y_test, y_pred_proba), 5),
            'cf_matrix': confusion_matrix(y_test, y_pred),
            'train_time': train_time_str,
            'notes': notes}

In [3]:
def store_test_result(result):
    test_results.loc[len(test_results)] = result

## Load data (final data file)

In [4]:
df = pd.read_csv('data/merged_data.csv')
df.shape

(447998, 44)

In [5]:
df['toxic'] = df['toxicity'] >= 0.5

In [6]:
df_new = df[['comment_text', 'toxic']].copy()

In [7]:
df_new

,comment_text,toxic
0,OH yes - Were those evil Christian Missionarie...,True
1,Why is this black racist crap still on the G&M...,True
2,even up here.......BLACKS!,True
3,Blame men. There's always an excuse to blame ...,True
4,And the woman exposing herself saying grab thi...,True
...,...,...
447993,Another man shamming article. If white men did...,False
447994,"""no matter what is put in front of you regardi...",False
447995,The Democrat party aided and abetted by it's M...,False
447996,I just don't find her a very good representati...,False


In [8]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447998 entries, 0 to 447997
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_text  447998 non-null  object
 1   toxic         447998 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 3.8+ MB


In [9]:
df = df_new

In [10]:
print('Checking for NaN\'s ...')
print(df.isna().sum())
rows_before = df.shape[0]
print("\nRows before dropping:", rows_before)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
rows_after = df.shape[0]
print('Rows after:', rows_after)
print('Rows dropped:', rows_before - rows_after)

Checking for NaN's ...
comment_text    0
toxic           0
dtype: int64

Rows before dropping: 447998
Rows after: 447998
Rows dropped: 0


In [11]:
df.head()

,comment_text,toxic
0,OH yes - Were those evil Christian Missionarie...,True
1,Why is this black racist crap still on the G&M...,True
2,even up here.......BLACKS!,True
3,Blame men. There's always an excuse to blame ...,True
4,And the woman exposing herself saying grab thi...,True


## Optional: Create smaller sample from data to speed up experiments

In [12]:
sample_size = None

# uncomment to create sample of desired size
#sample_size = 50_000

if sample_size != None:
    # ratio toxic/nontoxic
    tox_perc = 0.4
    nontox_perc = 0.6

    # number of toxic/nontoxic rows
    sample_size_tox = int(sample_size * tox_perc)
    sample_size_nontox = int(sample_size * nontox_perc)

    sample_tox = df[df['toxic'] == 1].sample(sample_size_tox,
                                             random_state=42)
    sample_nontox = df[df['toxic'] == 0].sample(sample_size_nontox,
                                                random_state=42)

    df = pd.concat([sample_tox, sample_nontox])
    print(f'Using sample ({df.shape[0]} rows).')

else:
    print(f'Using full data ({df.shape[0]} rows).')

Using full data (447998 rows).


## Create label/target variable and check for imbalance

In [13]:
target = df['toxic']

In [14]:
value_counts = target.value_counts()
nontoxic_count = value_counts[0]
toxic_count = value_counts[1]
nontoxic_perc =\
    round((nontoxic_count / (nontoxic_count + toxic_count)) * 100, 1)
toxic_perc =\
    round((toxic_count / (nontoxic_count + toxic_count)) * 100, 1)

print(f'Nontoxic (0): {nontoxic_count} ({nontoxic_perc} %)')
print(f'Toxic (1): {toxic_count} ({toxic_perc} %)')

Nontoxic (0): 397204 (88.7 %)
Toxic (1): 50794 (11.3 %)


## Function for bag of words

In [15]:
def bow(data):
    vect = CountVectorizer()
    return vect.fit_transform(data)

## Run baseline model (logistic regression) on different data cols

In [16]:
# parameters for model
params = {'max_iter': 2_000}

# load model with parameters
lr = LogisticRegression(**params)

test_result = test_model(lr, 'BASELINE (logistic regression)', params,
                    'bag of words on col "comment_text"', bow(df['comment_text']), target)
store_test_result(test_result)

## Show test results + total exec time

In [17]:
test_results

,model_name,model_params,data_desc,data_size,features_no,f1,acc,recall,prec,roc_auc,cf_matrix,train_time,notes
0,BASELINE (logistic regression),{'max_iter': 2000},"bag of words on col ""comment_text""",447998,162553,0.53912,0.91407,0.44325,0.6879,0.88588,"[[77398, 2043], [5656, 4503]]",1m 2s,


In [18]:
full_run_time = time.time() - full_run_time_start
print(f'Full run time: {int(full_run_time // 60)}m {round(full_run_time % 60)}s')

Full run time: 1m 15s
